Reference:
https://arxiv.org/pdf/1801.07698.pdf

https://github.com/lyakaap/Landmark2019-1st-and-3rd-Place-Solution/blob/master/src/modeling/metric_learning.py

https://www.kaggle.com/ragnar123/bert-baseline/comments

https://www.kaggle.com/musha2017/shopee-external-models

load the data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install bert-tensorflow==1.0.1
# downgrade bert to have gfile function

In [ ]:
!pip install tensorflow==2.3.0
# downgrade tensorflow --- some functions library changed

In [ ]:
from shutil import copyfile
import re
import os
import numpy as np
import pandas as pd
import random
import math
import matplotlib.pyplot as plt
import tensorflow as tf
import warnings
warnings.simplefilter('ignore')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import backend as K
t_path = '/content/drive/Shareddrives/deep learning project/data/shopee-external-models/tokenization.py'
copyfile(src = "/content/drive/Shareddrives/deep learning project/data/shopee-product-matching/shopee-external-models/tokenization.py", dst = "../content/tokenization.py")
from bert import tokenization
import tensorflow_hub as hub

In [ ]:
# set work dictionary
print('Working Directory')
print(os.getcwd())
work_dir = "/content/drive/Shareddrives/deep learning project/data/shopee-product-matching"

Working Directory
/content


In [ ]:
# import train and test in csv
test = pd.read_csv('/content/drive/Shareddrives/deep learning project/data/shopee-product-matching/test.csv')
train = pd.read_csv('/content/drive/Shareddrives/deep learning project/data/shopee-product-matching/train.csv')
# set image dataset path
test_paths = '/content/drive/Shareddrives/deep learning project/data/shopee-product-matching/test_images'
train_paths = '/content/drive/Shareddrives/deep learning project/data/shopee-product-matching/train_images'

In [ ]:
train.shape

(34250, 5)

According to the kaggle, image_phrash < image < postion id.

There are same pictures with different name;

There are repeat pictures.

In [ ]:
# add file path in the table (where to find the images)
train['path'] = '/content/drive/Shareddrives/deep learning project/data/shopee-product-matching/train_images/' + train['image']
test['path'] = '/content/drive/Shareddrives/deep learning project/data/shopee-product-matching/test_images/' + test['image']

### set f1 score

In [ ]:
def F1_score(target_column, pred_column): 
    def get_f1(row):
        # Find the common values in target and prediction arrays.
        intersection = len( np.intersect1d(row[target_column], row[pred_column]) )
        # Computes the score by following the formula
        f1_score = 2 * intersection / (len(row[target_column]) + len(row[pred_column]))
        return f1_score
    return get_f1
# reference: stackflow

### Text Embeddings on title (BERT)

In [ ]:
# Configuration
EPOCHS = 25
BATCH_SIZE = 32
# Seed
SEED = 123
# Verbosity
VERBOSE = 1
LR = 0.00001 # changed

In [ ]:
# Function to seed everything
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)
    
def read_and_preprocess():
    df = pd.read_csv('/content/drive/Shareddrives/deep learning project/data/shopee-product-matching/train.csv')
    tmp = df.groupby(['label_group'])['posting_id'].unique().to_dict()
    df['matches'] = df['label_group'].map(tmp)
    df['matches'] = df['matches'].apply(lambda x: ' '.join(x))
    encoder = LabelEncoder()
    df['label_group'] = encoder.fit_transform(df['label_group'])
    N_CLASSES = df['label_group'].nunique()
    print(f'We have {N_CLASSES} classes')
    x_train, x_val, y_train, y_val = train_test_split(df[['title']], df['label_group'], shuffle = True, stratify = df['label_group'], random_state = SEED, test_size = 0.33)
    return df, N_CLASSES, x_train, x_val, y_train, y_val

# Return tokens, masks and segments from a text array or series
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [ ]:
class ArcMarginProduct(tf.keras.layers.Layer):

    def __init__(self, n_classes, s=30, m=0.50, easy_margin=False,
                 ls_eps=0.0, **kwargs):

        super(ArcMarginProduct, self).__init__(**kwargs)

        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.ls_eps = ls_eps
        self.easy_margin = easy_margin
        self.cos_m = tf.math.cos(m)
        self.sin_m = tf.math.sin(m)
        self.th = tf.math.cos(math.pi - m)
        self.mm = tf.math.sin(math.pi - m) * m

    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'n_classes': self.n_classes,
            's': self.s,
            'm': self.m,
            'ls_eps': self.ls_eps,
            'easy_margin': self.easy_margin,
        })
        return config

    def build(self, input_shape):
        super(ArcMarginProduct, self).build(input_shape[0])

        self.W = self.add_weight(
            name='W',
            shape=(int(input_shape[0][-1]), self.n_classes),
            initializer='glorot_uniform',
            dtype='float32',
            trainable=True,
            regularizer=None)

    def call(self, inputs):
        X, y = inputs
        y = tf.cast(y, dtype=tf.int32)
        cosine = tf.matmul(
            tf.math.l2_normalize(X, axis=1),
            tf.math.l2_normalize(self.W, axis=0)
        )
        sine = tf.math.sqrt(1.0 - tf.math.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = tf.where(cosine > 0, phi, cosine)
        else:
            phi = tf.where(cosine > self.th, phi, cosine - self.mm)
        one_hot = tf.cast(
            tf.one_hot(y, depth=self.n_classes),
            dtype=cosine.dtype
        )
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.n_classes

        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s
        return output

# Function to build bert model
def build_bert_model(bert_layer, max_len = 512):
    
    margin = ArcMarginProduct(
            n_classes = N_CLASSES, 
            s = 30, 
            m = 0.5, 
            name='head/arc_margin', 
            dtype='float32'
            )
    
    input_word_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")
    label = tf.keras.layers.Input(shape = (), name = 'label')

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    x = margin([clf_output, label])
    output = tf.keras.layers.Softmax(dtype='float32')(x)
    model = tf.keras.models.Model(inputs = [input_word_ids, input_mask, segment_ids, label], outputs = [output])
    model.compile(optimizer = tf.keras.optimizers.Adam(lr = LR),
                  loss = [tf.keras.losses.SparseCategoricalCrossentropy()],
                  metrics = [tf.keras.metrics.SparseCategoricalAccuracy()])
    return model

def load_train_and_evaluate(x_train, x_val, y_train, y_val):
    seed_everything(SEED)
    # Load BERT from the Tensorflow Hub
    module_url = "/content/drive/Shareddrives/deep learning project/data/shopee-product-matching/shopee-external-models/bert_en_uncased_L-24_H-1024_A-16_1"
    bert_layer = hub.KerasLayer(module_url, trainable = True)
    vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
    do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
    tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)
    x_train = bert_encode(x_train['title'].values, tokenizer, max_len = 70)
    x_val = bert_encode(x_val['title'].values, tokenizer, max_len = 70)
    y_train = y_train.values
    y_val = y_val.values
    # Add targets to train and val
    x_train = (x_train[0], x_train[1], x_train[2], y_train)
    x_val = (x_val[0], x_val[1], x_val[2], y_val)
    bert_model = build_bert_model(bert_layer, max_len = 70)
    checkpoint = tf.keras.callbacks.ModelCheckpoint(f'Bert_{SEED}.h5', 
                                                    monitor = 'val_loss', 
                                                    verbose = VERBOSE, 
                                                    save_best_only = True,
                                                    save_weights_only = True, 
                                                    mode = 'min')
    history = bert_model.fit(x_train, y_train,
                             validation_data = (x_val, y_val),
                             epochs = EPOCHS, 
                             callbacks = [checkpoint],
                             batch_size = BATCH_SIZE,
                             verbose = VERBOSE)
    return bert_model,history
    

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
# use version 1 tensorflow

TensorFlow is already loaded. Please restart the runtime to change versions.


In [ ]:
df, N_CLASSES, x_train, x_val, y_train, y_val = read_and_preprocess()
# process data

We have 11014 classes


In [ ]:
#evaluate the train model
bert_model  = load_train_and_evaluate(x_train, x_val, y_train, y_val)

Epoch 1/25
718/718 [==============================] - ETA: 0s - loss: 23.9041 - sparse_categorical_accuracy: 0.0000e+00
Epoch 00001: val_loss improved from inf to 23.48550, saving model to Bert_123.h5
718/718 [==============================] - 690s 961ms/step - loss: 23.9041 - sparse_categorical_accuracy: 0.0000e+00 - val_loss: 23.4855 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 2/25
718/718 [==============================] - ETA: 0s - loss: 22.7323 - sparse_categorical_accuracy: 0.0000e+00
Epoch 00002: val_loss improved from 23.48550 to 22.67866, saving model to Bert_123.h5
718/718 [==============================] - 688s 958ms/step - loss: 22.7323 - sparse_categorical_accuracy: 0.0000e+00 - val_loss: 22.6787 - val_sparse_categorical_accuracy: 1.7694e-04
Epoch 3/25
718/718 [==============================] - ETA: 0s - loss: 21.3282 - sparse_categorical_accuracy: 0.0037
Epoch 00003: val_loss improved from 22.67866 to 21.88824, saving model to Bert_123.h5
718/718 [================

In [ ]:
bert_model

# Model Evaluation and Prediction

This notebook evaluates and makes predictions with models we trained.
1. CNN
2. ArcFace
3. Bert
4. ArcFace+Bert

References: https://www.kaggle.com/ragnar123/unsupervised-baseline-arcface

In [ ]:
import os 

# Set your working directory to a folder in your Google Drive. This way, if your notebook times out,
# your files will be saved in your Google Drive!

# the base Google Drive directory
root_dir = "/content/drive/Shareddrives/"

# choose where you want your project files to be saved
project_folder = "deep learning project/final_code"

def create_and_set_working_directory(project_folder):
  # check if your project folder exists. if not, it will be created.
  if os.path.isdir(root_dir + project_folder) == False:
    os.mkdir(root_dir + project_folder)
    print(root_dir + project_folder + ' did not exist but was created.')

  # change the OS to use your project folder as the working directory
  os.chdir(root_dir + project_folder)

  # create a test file to make sure it shows up in the right place
  !touch 'new_file_in_working_directory.txt'
  print('\nYour working directory was changed to ' + root_dir + project_folder + \
        "\n\nAn empty text file was created there. You can also run !pwd to confirm the current working directory." )

create_and_set_working_directory(project_folder)


Your working directory was changed to /content/drive/Shareddrives/deep learning project/final_code

An empty text file was created there. You can also run !pwd to confirm the current working directory.


In [ ]:
import re
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB3
from sklearn import metrics
#from tensorflow.keras import backend as K
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
import util

## Parameters

In [ ]:
TRAIN_CSV_PATH = '../../data/train.csv'
TRAIN_IMG_DIR = '../../data/train_images/'
ARCFACE_MODEL_PATH = './trained/arcface_best_epoch_512_42.h5'
CNN_MODEL_PATH = './trained/CNN.h5'
N_CLASSES = 11014
IMAGE_SIZE = [512,512]
BATCH_SIZE = 8
Bert_mdodel_path = '/content/drive/Shareddrives/deep learning project/final_code/bert_model'
AUTO = tf.data.experimental.AUTOTUNE

## Function Definition

In [ ]:
def get_dataset_csv():
    '''
    Read and prepare the tabular records
    '''
    train = pd.read_csv(TRAIN_CSV_PATH)
    tmp = train.groupby(['label_group'])['posting_id'].unique().to_dict()
    train['matches'] = train['label_group'].map(tmp)
    train['matches'] = train['matches'].apply(lambda x: ' '.join(x))
    img_paths = TRAIN_IMG_DIR + train['image']
        
    return train, img_paths

In [ ]:
def read_image(image):
    '''
    Parse an image
    '''
    image = tf.io.read_file(image)
    image = util.decode_image(image, IMAGE_SIZE)
    return image

# Function to get our dataset that read images
def get_dataset_img(image):
    '''
    Read and prepare the image dataset
    '''
    dataset = tf.data.Dataset.from_tensor_slices(image)
    dataset = dataset.map(read_image, num_parallel_calls = AUTO)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO)
    return dataset

In [ ]:
def load_model(which='arcface'):
    '''
    Load pretrained image models
    '''
    
    assert(which=='arcface' or which=='cnn'), 'which==\'arcface\' or \'cnn\''
    
    if which == 'cnn':
        print('Reminder: image size = (224,224)')
        return tf.keras.models.load_model('./trained/CNN.h5')
    
    inp = tf.keras.layers.Input(shape = (*IMAGE_SIZE, 3), name = 'inp1')
    label = tf.keras.layers.Input(shape = (), name = 'inp2')
    x = EfficientNetB3(weights = 'imagenet', include_top = False)(inp)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    
    margin = util.ArcMarginProduct(
        n_classes = N_CLASSES, 
        s = 30, 
        m = 0.5, 
        name='head/arc_margin', 
        dtype='float32'
    )
    
    x = margin([x, label])

    output = tf.keras.layers.Softmax(dtype='float32')(x)
    model = tf.keras.models.Model(inputs = [inp, label], outputs = [output])
    model.load_weights(ARCFACE_MODEL_PATH)
    model = tf.keras.models.Model(inputs = model.input[0], outputs = model.layers[-4].output)
    
    return model

In [ ]:
def embed_images(img_paths, model):
    image_dataset = get_dataset_img(img_paths)
    image_embeddings = model.predict(image_dataset)   
    return image_embeddings

In [ ]:
def embed_titles(df, max_features = 15500):
    tfidf = TfidfVectorizer(stop_words = 'english', binary = True, max_features = max_features)
    text_embeddings = tfidf.fit_transform(df['title'])
    return text_embeddings

In [ ]:
def nearest_neighbor(df, 
                     embeddings, 
                     thresholds = np.arange(0.3,0.6,0.01), 
                     n_neighbors = 50):
    '''
    1. Performs knn, then find the best threshold
    2. Use the best threshold to make predictions
    '''

    knn = NearestNeighbors(n_neighbors = n_neighbors, metric='cosine')
    knn.fit(embeddings)
    
    distances, indices = knn.kneighbors(embeddings)
    
    scores = []
    for threshold in thresholds:
        preds = []
        for k in range(embeddings.shape[0]):
            index_matches = np.where(distances[k,] < threshold)[0]
            id_matches = indices[k,index_matches]
            posting_ids = ' '.join(df['posting_id'].iloc[id_matches].values)
            preds.append(posting_ids)
        df['pred_matches'] = preds
        df['f1'] = util.f1_score(df['matches'], df['pred_matches'])
        score = df['f1'].mean()
        print(f'Threshold:{threshold} - F1: {score}')
        scores.append(score)
    df_thresholds = pd.DataFrame({'threshold':thresholds,'f1':scores})
    best_f1 = df_thresholds.f1.max()
    best_threshold = df_thresholds[df_thresholds.f1==best_f1].threshold.min()
    print('-'*100)
    print(f'The best threshold is {best_threshold} with an f1 of {best_f1}')
    
    best_preds = []
    for k in range(embeddings.shape[0]):
        index_matches = np.where(distances[k,] < best_threshold)[0]
        id_matches = indices[k,index_matches]
        posting_ids = df['posting_id'].iloc[id_matches].values
        best_preds.append(posting_ids)
    
    return df, best_threshold, best_f1, best_preds

## Evaluate and Predict

In [ ]:
df, img_paths = get_dataset_csv()

TypeError: ignored

In [ ]:
model = load_model('arcface')

43941888/43941136 [==============================] - 0s 0us/step


OSError: ignored

In [ ]:
# 00:06:00
image_embeddings = embed_images(img_paths, model)

Image embeddings shape is (34250, 1536)


In [ ]:
df, img_threshold, img_f1, img_preds = nearest_neighbor(df, 
                                                        image_embeddings, 
                                                        thresholds = np.arange(0.3,0.6,0.01))

Threshold:0.3 - F1: 0.807931808369746
Threshold:0.31 - F1: 0.815814757157693
Threshold:0.32 - F1: 0.8227750264643898
Threshold:0.33 - F1: 0.8296851575414907
Threshold:0.34 - F1: 0.836653371502557
Threshold:0.35000000000000003 - F1: 0.8435490339690283
Threshold:0.36000000000000004 - F1: 0.8497741344366871
Threshold:0.37000000000000005 - F1: 0.8565898347407793
Threshold:0.38000000000000006 - F1: 0.8629129501399706
Threshold:0.39000000000000007 - F1: 0.868926199591246
Threshold:0.4000000000000001 - F1: 0.8743222944555005
Threshold:0.4100000000000001 - F1: 0.8795463257153483
Threshold:0.4200000000000001 - F1: 0.8842590815368498
Threshold:0.4300000000000001 - F1: 0.8891689378633886
Threshold:0.4400000000000001 - F1: 0.8929264811939516
Threshold:0.4500000000000001 - F1: 0.8967264739132388
Threshold:0.46000000000000013 - F1: 0.9002619797067279
Threshold:0.47000000000000014 - F1: 0.9035653413378164
Threshold:0.48000000000000015 - F1: 0.9062124738966897
Threshold:0.49000000000000016 - F1: 0.908

In [ ]:
title_embeddings = bert_model(df)

TypeError: ignored

In [ ]:
df, text_threshold, title_f1, title_preds = nearest_neighbor(df, 
                                                      title_embeddings, 
                                                      thresholds = np.arange(0.3,0.6,0.01))

Threshold:0.3 - F1: 0.630776547251398
Threshold:0.31 - F1: 0.6352270544040849
Threshold:0.32 - F1: 0.6385704191694083
Threshold:0.33 - F1: 0.6425737224521486
Threshold:0.34 - F1: 0.6461641669803598
Threshold:0.35000000000000003 - F1: 0.6496528451842519
Threshold:0.36000000000000004 - F1: 0.653089439006744
Threshold:0.37000000000000005 - F1: 0.6557794983689451
Threshold:0.38000000000000006 - F1: 0.6579680043339512
Threshold:0.39000000000000007 - F1: 0.6598584657763932
Threshold:0.4000000000000001 - F1: 0.6616317472597671
Threshold:0.4100000000000001 - F1: 0.6635153361547791
Threshold:0.4200000000000001 - F1: 0.6651819166264529
Threshold:0.4300000000000001 - F1: 0.6659392906886799
Threshold:0.4400000000000001 - F1: 0.6660580341009082
Threshold:0.4500000000000001 - F1: 0.6656979203004432
Threshold:0.46000000000000013 - F1: 0.6644144768688384
Threshold:0.47000000000000014 - F1: 0.6638981909620011
Threshold:0.48000000000000015 - F1: 0.6619719121488122
Threshold:0.49000000000000016 - F1: 0.6

In [ ]:
def combine_predictions(row):
    '''
    Combines image predictions and text predictions (through union)
    '''
    x = np.union1d(row['pred_img'], row['pred_title'])
    return ' '.join(x)

In [ ]:
df['pred_img'] = img_preds
df['pred_title'] = title_preds
final_pred = df.apply(combine_predictions, axis=1)

In [ ]:
df['f1'] = util.f1_score(df['matches'], final_pred)
score = df['f1'].mean()
print(f'ArcFace+tfidf f1: {score}')

ArcFace+tfidf f1: 0.8397749437764472
